In [2]:
# Load packages and data
using DelimitedFiles, LinearAlgebra, Statistics, LeastSquaresOptim, Printf, Test, SparseArrays, Base

data = readdlm("houseprice.csv", ',', Float64)
x = data[:, 1:end .!= 1]
y = data[:, setdiff(1:end, (2,3))]
println("Done loading")

Done loading


#### Nonlinear Comparison: Dog Leg vs Levenberg–Marquardt

In [38]:
## Setting up F function and Jacobian

n = trunc(Int, size(x,1))

function watson(n::Integer)
    name = "watson"
    c9 = 2.9e1

    function f!(fvec, x)
        fill!(fvec, 0)
        for i = 1:29
            ti = i/c9
            sum1 = 0.0
            temp = 1.0
            for j = 2:n
                sum1 += (j-1)*temp*x[j]
                temp *= ti
            end
            sum2 = 0.0
            temp = 1.0
            for j = 1:n
                sum2 += temp*x[j]
                temp *= ti
            end
            temp1 = sum1 - sum2^2 - 1
            temp2 = 2*ti*sum2
            temp = 1/ti
            for k = 1:n
                fvec[k] += temp*(k-1-temp2)*temp1
                temp *= ti
            end
        end
        temp = x[2] - x[1]^2 - 1
        fvec[1] += x[1]*(1-2temp)
        fvec[2] += temp
    end

    function g!(fjac, x)
        fill!(fjac, 0)
        for i = 1:29
            ti = i/c9
            sum1 = 0.0
            temp = 1.0
            for j = 2:n
                sum1 += (j-1)*temp*x[j]
                temp *= ti
            end
            sum2 = 0.0
            temp = 1.0

            for j = 1:n
                sum2 += temp*x[j]
                temp *= ti
            end
            temp1 = 2(sum1 - sum2^2 - 1)
            temp2 = 2*sum2
            temp = ti^2
            tk = 1.0

            for k = 1:n
                tj = tk
                for j = k:n
                    fjac[k,j] += tj*((Float64(k-1)/ti - temp2)*(Float64(j-1)/ti - temp2) - temp1)
                    tj *= ti
                end
                tk *= temp
            end
        end
        fjac[1,1] += 6x[1]^2 - 2x[2] + 3
        fjac[1,2] -= 2x[1]
        fjac[2,2] += 1
        for k = 1:n
            for j = k:n
                fjac[j,k] = fjac[k,j]
            end
        end
    end
    x = zeros(n)
    return name, f!, g!, x
end

watson (generic function with 1 method)

In [39]:
iter = 0
@printf("method   optimizer     iter  f_calls  g_calls             RMSD\n")
for (optimizer, optimizer_abbr) in ((LeastSquaresOptim.Dogleg, :DogLeg), (LeastSquaresOptim.LevenbergMarquardt, :LevMar))
        global iter += 1
        global f!, g! = watson(n::Integer)
        global J = ones(365, 365)
        g!(J, x)
        J = sparse(J)
        global nls = LeastSquaresProblem(x = x, y = y, f! = f!, J = J, g! = g!)
        global r = optimize!(nls, optimizer(LeastSquaresOptim.LSMR()))
        if iter == 1
        end
        @printf("%6s %11s %8d %8d %8d  %15e\n", solver_abbr, optimizer_abbr, r.iterations, r.f_calls, r.g_calls, ((r.ssr)/size(x,1))^(0.5))
        @test r.ssr <= 50000
        @test r.converged
end

method   optimizer     iter  f_calls  g_calls             RMSD


LoadError: MethodError: objects of type String are not callable